# CNN_Platanos_Calidad
Importa las bibliotecas necesarias para el procesamiento de imágenes, la creación y entrenamiento de modelos de redes neuronales, y la gestión de conjuntos de datos.

- os: Proporciona funciones para interactuar con el sistema operativo, como manipulación de rutas de archivos y directorios.
- cv2: Biblioteca OpenCV para procesamiento de imágenes y visión por computadora.
- numpy: Biblioteca para cálculos numéricos eficientes en Python.
- torch: Biblioteca principal de PyTorch para cálculos en tensores y operaciones en redes neuronales.
- torch.nn: Módulo de PyTorch que proporciona herramientas para la definición y entrenamiento de redes neuronales.
- torch.optim: Módulo de PyTorch que contiene implementaciones de varios optimizadores para ajustar los pesos de los modelos.
- torch.utils.data: Utilidades para trabajar con conjuntos de datos en PyTorch.
- Dataset: Clase base para definir conjuntos de datos personalizados.
- DataLoader: Clase que permite cargar datos en lotes y proporciona funcionalidad para el entrenamiento en lotes.
- torchvision.transforms.ToTensor: Transforma una imagen PIL o NumPy en un tensor de PyTorch.

No se pasan parámetros a estas importaciones, simplemente se importan las bibliotecas necesarias para el código posterior.

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

In [ ]:
# Definir la clase del conjunto de datos personalizado
class BananaDataset(Dataset):
    def __init__(self, root_dir):
        """
        Clase personalizada para el conjunto de datos de plátanos de diferentes calidades.

        Parámetros:
        - root_dir (str): Ruta del directorio raíz que contiene las carpetas de calidad de las imágenes.
        """
        self.root_dir = root_dir
        self.quality_folders = ['excelente', 'regular', 'baja', 'mala']
        self.images = []
        self.labels = []

        # Recorrer las carpetas de calidad y cargar las imágenes y etiquetas correspondientes
        for i, quality in enumerate(self.quality_folders):
            folder_path = os.path.join(self.root_dir, quality)
            image_files = os.listdir(folder_path)

            for image_file in image_files:
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (512, 512))
                self.images.append(image)
                self.labels.append(i)

    def __len__(self):
        """
        Retorna la cantidad de imágenes en el conjunto de datos.

        Retorna:
        - length (int): Cantidad de imágenes en el conjunto de datos.
        """
        return len(self.images)

    def __getitem__(self, idx):
        """
        Retorna una imagen y su etiqueta correspondiente en la posición `idx`.

        Parámetros:
        - idx (int): Índice de la imagen y etiqueta a obtener.

        Retorna:
        - image (Tensor): Imagen representada como un tensor.
        - label (int): Etiqueta de calidad de la imagen.
        """
        image = self.images[idx]
        label = self.labels[idx]
        return ToTensor()(image), label


Crea una instancia del conjunto de datos personalizado llamado `BananaDataset`, 
que se utiliza para cargar y gestionar las imágenes de plátanos de diferentes calidades.

Parámetros:
- 'img' (str): Ruta del directorio raíz que contiene las carpetas de calidad de las imágenes.

Retorna:
- dataset (BananaDataset): Una instancia del conjunto de datos personalizado.

In [ ]:
# Crear una instancia del conjunto de datos
dataset = BananaDataset('img')

# Dividir los datos en conjuntos de entrenamiento y prueba
train_ratio = 0.8
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size

Divide el conjunto de datos en conjuntos de entrenamiento y prueba, con una proporción especificada.

Parámetros:
- train_ratio (float): Proporción de datos a asignar al conjunto de entrenamiento.
- len(dataset) (int): Cantidad total de datos en el conjunto de datos.
- train_size (int): Cantidad de datos asignados al conjunto de entrenamiento.
- test_size (int): Cantidad de datos asignados al conjunto de prueba.

Retorna:
- train_set (Subset): Conjunto de datos de entrenamiento.
- test_set (Subset): Conjunto de datos de prueba.

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

Crea cargadores de datos para el conjunto de entrenamiento y prueba.

Parámetros:
- train_set (Subset): Conjunto de datos de entrenamiento.
- test_set (Subset): Conjunto de datos de prueba.
- batch_size (int): Tamaño del lote (número de muestras por lote).
- shuffle (bool): Indica si se deben mezclar los datos en cada época o no.

Retorna:
- train_loader (DataLoader): Cargador de datos para el conjunto de entrenamiento.
- test_loader (DataLoader): Cargador de datos para el conjunto de prueba.

In [ ]:

# Crear cargadores de datos para el conjunto de entrenamiento y prueba
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

 Clase que define un modelo de red neuronal convolucional (CNN).

Atributos:
- conv1 (nn.Conv2d): Capa de convolución 2D inicial.
- pool (nn.MaxPool2d): Capa de pooling para reducción de dimensionalidad.
- conv2 (nn.Conv2d): Capa de convolución 2D adicional.
- fc1 (nn.Linear): Capa totalmente conectada (fully connected).
- fc2 (nn.Linear): Capa totalmente conectada final que produce las salidas.


In [ ]:
# Definir el modelo de CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 127 * 127, 64)
        self.fc2 = nn.Linear(64, 4)  # 4 clases de calidad en total

    def forward(self, x):
        """
        Implementación del paso hacia adelante (forward pass) del modelo de CNN.

        Parámetros:
        - x (Tensor): Datos de entrada (imágenes) representados como un tensor.

        Retorna:
        - x (Tensor): Salida del modelo.

        """
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 127 * 127)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Crea una instancia del modelo de la red neuronal convolucional (CNN, por sus siglas en inglés) denominado `CNN`. 
Este modelo ha sido diseñado específicamente para el reconocimiento de calidad de plátanos y clasificación en diferentes categorías de calidad.

Retorna:
- model (CNN): Una instancia del modelo de la red neuronal convolucional para el reconocimiento de calidad de plátanos.


Define la función de pérdida utilizada para entrenar el modelo. En este caso, se utiliza la función de pérdida 
de entropía cruzada (CrossEntropyLoss), que es adecuada para problemas de clasificación multiclase.

Retorna:
- criterion (CrossEntropyLoss): La función de pérdida de entropía cruzada.


Crea el optimizador utilizado para ajustar los pesos del modelo durante el entrenamiento. Se utiliza el algoritmo Adam, 
que es un método de optimización popular y eficiente para el entrenamiento de redes neuronales.

Parámetros:
- model.parameters(): Parámetros del modelo que se optimizarán.
- lr (float): Tasa de aprendizaje, que determina la velocidad de ajuste de los pesos durante el entrenamiento.

Retorna:
- optimizer (Adam): El optimizador Adam configurado con los parámetros del modelo y la tasa de aprendizaje especificada.

In [ ]:
# Crear una instancia del modelo y definir la función de pérdida y el optimizador
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Número de épocas, que representa la cantidad de veces que el modelo recorre todo el conjunto de datos de entrenamiento durante el proceso de entrenamiento.

Parámetros:
- num_epochs (int): Número de épocas para entrenar el modelo.

Detecta si una GPU (CUDA) está disponible y configura el dispositivo a utilizar. 
Si CUDA está disponible, el dispositivo se establece en "cuda", de lo contrario, se establece en "cpu".

Retorna:
- device (torch.device): El dispositivo utilizado para el entrenamiento (GPU o CPU).

Mueve el modelo a la GPU o la CPU según el dispositivo detectado.

Parámetros:
- device (torch.device): El dispositivo al que se moverá el modelo (GPU o CPU).

Por ultimo: Realiza el bucle de entrenamiento del modelo durante el número de épocas especificado. En cada época, se itera sobre los lotes de datos del conjunto de entrenamiento y se realiza la propagación hacia adelante, el cálculo de la pérdida, la propagación hacia atrás y la actualización de los pesos del modelo.

Imprime el valor de pérdida promedio para cada época.

No se retornan valores, solo se entrena el modelo y se imprime el progreso del entrenamiento.

In [ ]:
# Entrenar el modelo
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader)}")

Pone el modelo en modo de evaluación. Esto es necesario para desactivar ciertas capas como la capa de dropout que se utilizan durante el entrenamiento pero no durante la evaluación.

No se pasan parámetros y no hay valor de retorno.

Variables para realizar un seguimiento del número de 
predicciones correctas y el número total de imágenes evaluadas.

- correct (int): Número de predicciones correctas.
- total (int): Número total de imágenes evaluadas.

Evalúa el modelo utilizando el conjunto de prueba. Para cada lote de imágenes y etiquetas en el conjunto de prueba, se realizan predicciones utilizando el modelo y se comparan con las etiquetas verdaderas.

- images (torch.Tensor): Tensores de imágenes del lote.
- labels (torch.Tensor): Tensores de etiquetas del lote.

No se retornan valores, las variables correct y total se actualizan con el conteo de predicciones correctas y el número total de imágenes evaluadas.

Calcula la precisión del modelo en el conjunto de prueba como un porcentaje.

Retorna:
- accuracy (float): Precisión del modelo en el conjunto de prueba.

Imprime la precisión del modelo en el conjunto de prueba.

No se retorna ningún valor.

In [ ]:
# Evaluar el modelo
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total


print(f"Accuracy on test set: {accuracy}%")